# Simulating with FBA

Simulations using flux balance analysis can be solved using Model.optimize(). This will maximize or minimize (maximizing is the default) flux through the objective reactions.

In [12]:
import pandas
pandas.options.display.max_rows = 100

import cobra.test
model = cobra.test.create_test_model("textbook")

## Running FBA

In [2]:
model.optimize()

<Solution 0.87 at 0x7fb3f849ca10>

The Model.optimize() function will return a Solution object, which will also be stored at model.solution. A solution object has several attributes:

 - f: the objective value
 - status: the status from the linear programming solver
 - x_dict: a dictionary of {reaction_id: flux_value} (also called "primal")
 - x: a list for x_dict
 - y_dict: a dictionary of {metabolite_id: dual_value}.
 - y: a list for y_dict

For example, after the last call to model.optimize(), the status should be 'optimal' if the solver returned no errors, and f should be the objective value

In [3]:
model.solution.status

'optimal'

In [4]:
model.solution.f

0.8739215069684305

## Changing the Objectives

The objective function is determined from the objective_coefficient attribute of the objective reaction(s). Currently in the model, there is only one objective reaction, with an objective coefficient of 1.

In [5]:
model.objective

{<Reaction Biomass_Ecoli_core at 0x7fb3c899be90>: 1.0}

The objective function can be changed by assigning Model.objective, which can be a reaction object (or just it's name), or a dict of {Reaction: objective_coefficient}.

In [6]:
# change the objective to ATPM
# the upper bound should be 1000 so we get the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.
model.objective = "ATPM"
model.objective

{<Reaction ATPM at 0x7fb3c899bbd0>: 1}

In [7]:
model.optimize()

<Solution 175.00 at 0x7fb3c895de50>

The objective function can also be changed by setting Reaction.objective_coefficient directly.

In [9]:
model.reactions.get_by_id("ATPM").objective_coefficient = 0.
model.reactions.get_by_id("Biomass_Ecoli_core").objective_coefficient = 1.
model.objective

{<Reaction Biomass_Ecoli_core at 0x7fb3c899be90>: 1.0}

## Running FVA

FBA will not give always give unique solution, because multiple flux states can achieve the same optimum. FVA (or flux variability analysis) finds the ranges of each metabolic flux at the optimum.

In [35]:
fva_result = cobra.flux_analysis.flux_variability_analysis(model, model.reactions[:20])
pandas.DataFrame.from_dict(fva_result).T

,maximum,minimum
ACALD,9.466331e-29,3.720797e-15
ACALDt,-6.310887e-29,3.720797e-15
ACKr,-2.524355e-28,3.933509e-15
ACONTa,6.007250e+00,6.007250e+00
ACONTb,6.007250e+00,6.007250e+00
ACt2r,6.121561e-28,3.933509e-15
ADK1,-4.042971e-14,0.000000e+00
AKGDH,5.064376e+00,5.064376e+00
AKGt2r,0.000000e+00,7.079399e-15
ALCD2x,0.000000e+00,5.729185e-15


Setting parameter fraction_of_optimium=0.90 would give the flux ranges for reactions at 90% optimality.

In [36]:
fva_result = cobra.flux_analysis.flux_variability_analysis(model, model.reactions[:20], fraction_of_optimum=0.9)
pandas.DataFrame.from_dict(fva_result).T

,maximum,minimum
ACALD,9.466331e-29,-2.542370
ACALDt,-6.310887e-29,-2.542370
ACKr,-3.029226e-28,-3.813556
ACONTa,8.894520e+00,0.848587
ACONTb,8.894520e+00,0.848587
ACt2r,3.407879e-28,-3.813556
ADK1,1.716100e+01,0.000000
AKGDH,8.045934e+00,0.000000
AKGt2r,0.000000e+00,-1.430083
ALCD2x,0.000000e+00,-2.214323
